# Test Notebook
For testing models to be used to identify duplication resume form questions

In [1]:
from gensim.models import FastText
from gensim.test.utils import common_texts
import gensim.downloader
from nltk.tokenize.regexp import regexp_tokenize
from nltk import word_tokenize
import pandas as pd
import swifter
import string

In [2]:
from nltk.corpus import stopwords
import numpy as np
from scipy.spatial.distance import euclidean, cosine
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [63]:
from sklearn.metrics import accuracy_score, f1_score, log_loss
import lightgbm as lgb
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import precision_recall_curve
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

In [ ]:
model = gensim.downloader.load('fasttext-wiki-news-subwords-300')

In [ ]:
bmodel = SentenceTransformer('quora-distilbert-base')

In [4]:
sw_set = set(stopwords.words('english'))

In [5]:
train_df = pd.read_csv('./data/train.csv')
sample_df = train_df.dropna()


In [6]:
table = str.maketrans('/', ' ', '\'[\\]^_`{|}~"#$%&()*+,-:;<=>@?.!' + string.digits)
def clean_chars(text):
    text = text.replace('\t', ' ')
    text = text.translate(table)
    return text.strip()

In [7]:
def clean_text(text):
    text = clean_chars(text)
    words = (x.lower() for x in word_tokenize(text))
    words = [word for word in words if word not in sw_set]
    return words

In [8]:
clean_text('How can I be a good geologist?')
# kind of aggressive, but good for our use case

['good', 'geologist']

In [209]:
ss = [
    'Are you authorized to work in the United States without Visa Sponsorship or EAD?',
    'Will you now, or in the future, require sponsorship for employment Visa status (e.g., H-1B, visa status)?',
    'Will you now or in the future require sponsorship for employment visa status (e.g. H1B status)?',
    'How did you hear about this job?',
    'What was your favorite technical question that you were asked during an interview?',
    'Why are you interested in working at Karat?',
    'Are you legally eligible to work in the U.S.?',
    'Will you now or in the future require visa sponsorship for employment at the Karat?',
    'First Name',
    'Last Name',
    'Show us your work! Please share any links to personal, open source projects or code samples. The team would love to see your work.'
]

In [210]:
for i in range(len(ss)):
    print('='*25)
    for j in range(i+1, len(ss)):
        if i == j:
            continue
        print(ss[i], 'vs', ss[j], ':', jaccard(clean_text(ss[i]), clean_text(ss[j])))

Are you authorized to work in the United States without Visa Sponsorship or EAD? vs Will you now, or in the future, require sponsorship for employment Visa status (e.g., H-1B, visa status)? : 0.14285714285714285
Are you authorized to work in the United States without Visa Sponsorship or EAD? vs Will you now or in the future require sponsorship for employment visa status (e.g. H1B status)? : 0.14285714285714285
Are you authorized to work in the United States without Visa Sponsorship or EAD? vs How did you hear about this job? : 0.0
Are you authorized to work in the United States without Visa Sponsorship or EAD? vs What was your favorite technical question that you were asked during an interview? : 0.0
Are you authorized to work in the United States without Visa Sponsorship or EAD? vs Why are you interested in working at Karat? : 0.0
Are you authorized to work in the United States without Visa Sponsorship or EAD? vs Are you legally eligible to work in the U.S.? : 0.09090909090909091
Are 

In [188]:
w1 = clean_text('Are you authorized to work in the United States without Visa Sponsorship or EAD?')

In [189]:
w2 = clean_text('Will you now, or in the future, require sponsorship for employment Visa status (e.g., H-1B, visa status)?')

In [190]:
w3 = clean_text('Will you now or in the future require sponsorship for employment visa status (e.g. H1B status)?')

In [192]:
w4 = clean_text('How did you hear about this job?')
# f4 = extract_feats(w4)

In [202]:
jaccard(w2, w1)

0.14285714285714285

In [ ]:
w1

In [ ]:
w4

In [ ]:
f1 = extract_feats(w1)

In [ ]:
f2 = extract_feats(w2)

In [ ]:
f3 = extract_feats(w3)

In [ ]:
testSamp = feat_vector(f1, f2)
bst.predict(testSamp.reshape(1, -1))

In [ ]:
testSamp = feat_vector(f1, f3)
bst.predict(testSamp.reshape(1, -1))

In [ ]:
testSamp = feat_vector(f1, f4)
bst.predict(testSamp.reshape(1, -1))

In [ ]:
testSamp.shape

try bag of word

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from itertools import chain

In [135]:
vectorizer = CountVectorizer(stop_words='english', max_features=3000)

In [136]:
mat = vectorizer.fit_transform(chain(sample_df.question1, sample_df.question2))

In [137]:
np.abs(vectorizer.transform([ss[0]]) - vectorizer.transform([ss[1]])).shape

(1, 3000)

In [149]:
lr.predict_proba(np.abs(vectorizer.transform([ss[1]]) - vectorizer.transform([ss[2]])))

array([[0.68128587, 0.31871413]])

In [116]:
nb.predict(np.abs(vectorizer.transform([ss[1]]) - vectorizer.transform([ss[2]])))

array([0])

In [152]:
sample_df['q1_words'] = sample_df.question1.apply(clean_text)

<ipython-input-152-a381a184eda1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['q1_words'] = sample_df.question1.apply(clean_text)


In [153]:
sample_df['q2_words'] = sample_df.question2.apply(clean_text)

<ipython-input-153-fec924e97a31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['q2_words'] = sample_df.question2.apply(clean_text)


In [186]:
def jaccard(a, b):
    a_s = set(a)
    b_s = set(b)
    if len(a_s) == 0 and len(b_s) == 0:
        return 1
    return len(a_s & b_s) / len(a_s | b_s)

In [187]:
jacc = []
for (a, b) in zip(sample_df.q1_words, sample_df.q2_words):
    jacc.append(jaccard(a,b))

array([[0.83333333],
       [0.22222222],
       [0.22222222],
       ...,
       [0.33333333],
       [0.        ],
       [1.        ]])

In [155]:
sample_df.q2_words

0                [step, step, guide, invest, share, market]
1         [would, happen, indian, government, stole, koh...
2                [internet, speed, increased, hacking, dns]
3                    [find, remainder, math, math, divided]
4                       [fish, would, survive, salt, water]
                                ...                        
404285    [many, keywords, perl, programming, language, ...
404286                                  [true, life, death]
404287                                        [whats, coin]
404288    [little, hairfall, problem, want, use, hair, s...
404289                                  [like, sex, cousin]
Name: q2_words, Length: 404287, dtype: object

In [48]:
vectorizer.stop_words_

set()

In [139]:
mat_q1 = vectorizer.transform(sample_df.question1)
mat_q2 = vectorizer.transform(sample_df.question2)

In [140]:
diff_mat = np.abs(mat_q1 - mat_q2)

In [167]:
diff_mat

<404287x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 1594206 stored elements in Compressed Sparse Row format>

In [168]:
np.array(jacc).reshape(-1, 1).shape

(404287, 1)

In [169]:
from scipy import sparse

In [170]:
diff_mat = sparse.hstack([diff_mat, np.array(jacc).reshape(-1, 1)])

In [75]:
lr = LogisticRegression(verbose=1)

In [141]:
from sklearn.naive_bayes import BernoulliNB

In [113]:
nb = BernoulliNB()

In [180]:
nb.fit(trainX, trainY)

BernoulliNB()

In [171]:
trainX, testX, trainY, testY = train_test_split(diff_mat, sample_df.is_duplicate)

In [172]:
lr.fit(trainX, trainY)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/will/anaconda3/envs/scale-new/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s finished


LogisticRegression(verbose=1)

In [179]:
predY = lr.predict(testX)

In [181]:
predY = nb.predict(testX)

In [182]:
accuracy_score(testY, predY)

0.7057246319455438

In [183]:
f1_score(testY, predY)

0.6545087061064713

In [184]:
log_loss(testY, predY)

10.164074532997779

In [53]:
mat_q1.shape

(404287, 85846)

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
len(sample_df)

404287

In [ ]:
len(sample_df) // 5 / 2.42 / 60 / 60

In [ ]:
q1_chunks = chunks(list(sample_df['question1']), 5)
out = []
for q1_chunk in tqdm(q1_chunks):
    out.append(bmodel.encode(q1_chunk))
# sample_df['bert2']

In [ ]:
sample_df['q1_words'] = sample_df.question1.swifter.apply(clean_text)
sample_df['q2_words'] = sample_df.question2.swifter.apply(clean_text)

In [ ]:
def extract_feats(words):
    vecs = []
    for word in words:
        if word in model:
            vecs.append(model[word]) # hopefully no overflow
    if not vecs:
        return np.nan
    mat = np.vstack(vecs)
    return mat.mean(axis=0)
#     vec = np.zeros(300,)
#     cnt = 0
#     for word in words:
#         if word in model:
#             vec += model[word] # hopefully no overflow
#             cnt += 1
#     vec /= cnt # not always eql to len(words)
#     return vec

In [ ]:
sample_df['q1_feats'] = sample_df.q1_words.swifter.apply(extract_feats)
sample_df['q2_feats'] = sample_df.q2_words.swifter.apply(extract_feats)

In [ ]:
def feat_vector(a, b):
    out = a - b
#     out = np.hstack([a, b])
    more = np.array([euclidean(a, b), cosine(a, b)])
    return np.hstack([out, more])
#     return more

In [ ]:
good_els1 = sample_df['q1_feats'].swifter.apply(lambda x: isinstance(x, np.ndarray) and not np.isnan(x).any())
good_els2 = sample_df['q2_feats'].swifter.apply(lambda x: isinstance(x, np.ndarray) and not np.isnan(x).any())
good_els = good_els1 & good_els2

In [ ]:
good_df = sample_df[good_els]

In [ ]:
good_df['diff'] = good_df.swifter.apply(lambda x: feat_vector(x['q1_feats'], x['q2_feats']), axis=1)

In [ ]:
good_df = good_df.dropna()

In [ ]:
lr = LogisticRegression(verbose=1)
rf = RandomForestClassifier()

In [ ]:
len(good_df['diff'])

In [ ]:
len(good_df.is_duplicate)

In [ ]:
y.sum()

In [ ]:
X = np.vstack(list(good_df['diff']))
y = np.array(good_df.is_duplicate)

In [ ]:
X_train, X_other, y_train, y_other = train_test_split(X, y, train_size=0.8)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, train_size=0.3)

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)

In [ ]:
val_data = lgb.Dataset(X_val, label=y_val)

In [ ]:
bst = lgb.train({}, train_data, 100, valid_sets=[val_data])

In [ ]:
y_pred = bst.predict(X_test)

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)

In [ ]:
np.argsort(np.vstack([precision, recall]).sum(axis=0))[::-1][:10]

In [ ]:
log_loss(y_test, y_pred)

In [ ]:
vals = np.arange(0, 1, 0.01)
out = []
for v in tqdm(vals):
    out.append(f1_score(y_test, y_pred >= v))
plt.plot(vals, out)

In [ ]:
vals = np.arange(0, 1, 0.001)
out = []
for v in tqdm(vals):
    out.append(accuracy_score(y_test, y_pred >= v))
plt.plot(vals, out)

In [ ]:
X_train.shape[0]

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf_pred = rf.predict(X_test)

In [ ]:
accuracy_score(rf_pred, y_test)

In [ ]:
f1_score(rf_pred, y_test)

In [ ]:
rf.predict(X_test).sum()

In [ ]:
lr.predict(X_train).sum()

In [ ]:
# sample_df[sample_df.is_duplicate == 1]

In [ ]:
regexp_tokenize('I like to eat pie', pattern='')

In [ ]:
tiny_df = sample_df.sample(10000)

In [ ]:
q1_bert = bmodel.encode(list(tiny_df.question1), show_progress_bar=True)

In [ ]:
q2_bert = bmodel.encode(list(tiny_df.question2), show_progress_bar=True)

In [ ]:
sX = q1_bert - q2_bert

In [ ]:
sY = tiny_df.is_duplicate

In [ ]:
train_Sx, test_Sx, train_sY, test_sY = train_test_split(sX, sY)

In [ ]:
tiny_data = lgb.Dataset(train_Sx, label=train_sY)

In [ ]:
bst = lgb.train({}, tiny_data, 100)

In [ ]:
log_loss(test_sY, bst.predict(test_Sx))